In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f

spark = SparkSession.builder.master("local[1]") \
                    .appName('py_spark') \
                    .getOrCreate()


csvDF =spark.read.option("header",True).csv("/home/james/python_practice/archive/CAvideos.csv")

jsonDF =spark.read.option("multiLine", True).json("/home/james/python_practice/archive/CA_category_id.json").rdd.flatMap(lambda x:x["items"]).toDF()

joinDF=csvDF.join(jsonDF, csvDF['category_id'] == jsonDF['id'] )

cast_df =joinDF.withColumn("dislikes",  col('dislikes').cast(IntegerType()))
cast_df=joinDF.select(joinDF["dislikes"],joinDF['snippet.title'])
cast_df.show()
final_df =cast_df.groupBy('title').agg(f.sum('dislikes').alias('dislikes')).sort(f.col('dislikes').desc()).head(10)
spark.createDataFrame(final_df).show()
# joinDF.show(100)

21/10/18 15:16:06 WARN Utils: Your hostname, james-MS-7A69 resolves to a loopback address: 127.0.1.1; using 10.100.0.179 instead (on interface enp0s31f6)
21/10/18 15:16:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/18 15:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+--------+----------------+
|dislikes|           title|
+--------+----------------+
|    1204|Film & Animation|
|     348|Film & Animation|
|    2520|Film & Animation|
|      84|Film & Animation|
|      27|Film & Animation|
|     831|Film & Animation|
|   60025|Film & Animation|
|    1985|Film & Animation|
|    3657|Film & Animation|
|    5508|Film & Animation|
|     648|Film & Animation|
|    8506|Film & Animation|
|     569|Film & Animation|
|     224|Film & Animation|
|    8927|Film & Animation|
|      60|Film & Animation|
|     543|Film & Animation|
|     956|Film & Animation|
|    1135|Film & Animation|
|   51293|Film & Animation|
+--------+----------------+
only showing top 20 rows



+--------------------+-----------+
|               title|   dislikes|
+--------------------+-----------+
|       Entertainment|2.5791583E7|
|               Music| 2.209819E7|
|      People & Blogs|  9480796.0|
|              Comedy|  5372515.0|
|              Sports|  3286369.0|
|     News & Politics|  3264041.0|
|    Film & Animation|  3111259.0|
|              Gaming|  2446882.0|
|Science & Technology|  1791153.0|
|       Howto & Style|  1730520.0|
+--------------------+-----------+



In [5]:
bigger_df=joinDF[joinDF['dislikes'] >1000]
bigger_df = bigger_df['title','dislikes']
bigger_df.show()

+--------------------+--------+
|               title|dislikes|
+--------------------+--------+
|WE ARE ALL GOING ...|    3035|
|WE ARE ALL GOING ...|    3322|
|WE ARE ALL GOING ...|    3473|
|STUNG by a WARRIO...|    1486|
|STUNG by a WARRIO...|    2433|
|STUNG by a WARRIO...|    2754|
|STUNG by a WARRIO...|    2932|
|STUNG by a WARRIO...|    3042|
|Dog Cries Every T...|    1224|
|Heart-Wrenching V...|    1455|
|BITTEN by a GIANT...|    1646|
|BITTEN by a GIANT...|    2354|
|VENOM EXTRACTION ...|    1701|
|VENOM EXTRACTION ...|    2391|
|VENOM EXTRACTION ...|    2745|
|VENOM EXTRACTION ...|    2981|
|VENOM EXTRACTION ...|    3182|
|The Smallest Bird...|    1015|
|The Smallest Bird...|    1581|
|The Smallest Bird...|    1827|
+--------------------+--------+
only showing top 20 rows

